<a href="https://colab.research.google.com/github/tripathi-Krishna17/Mental-Health-Analysis/blob/main/Mental.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ==========================================
# 1. SETUP & INSTALLATION
# ==========================================

In [ ]:

!pip install datasets transformers xgboost scikit-learn accelerate torch -q

import pandas as pd
import numpy as np
import re
import torch
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ==========================================
# 2. DATA LOADING & PREPROCESSING
# ==========================================

In [ ]:

print("Loading dataset...")
# Loading the dataset from Hugging Face
dataset = load_dataset("solomonk/reddit_mental_health_posts")

# Convert to Pandas DataFrame for easier manipulation
df = dataset['train'].to_pandas()

In [ ]:
df.head()

In [ ]:
# --- Data Cleaning ---
print("Cleaning data...")

# 1. Filter necessary columns (assuming structure based on reddit datasets)
# Inspecting columns to ensure we pick the right ones
print(f"Available columns: {df.columns}")
# Usually 'title', 'body', and 'subreddit' are key.
# We will fill NaNs and combine title + body for richer context.
df['title'] = df['title'].fillna('')
df['body'] = df['body'].fillna('')
df['text'] = df['title'] + " " + df['body']

In [ ]:
# 2. Filter for top classes to ensure model stability (Multi-class classification)
# Let's focus on the top 5 most frequent subreddits to make the demo robust and runnable
top_n_classes = 5
top_subreddits = df['subreddit'].value_counts().nlargest(top_n_classes).index
df = df[df['subreddit'].isin(top_subreddits)]

In [ ]:
# 3. Sampling (Optional but recommended for Colab speed)
# Using 2000 samples for demonstration. Comment this out to use the full dataset.
df = df.sample(n=2000, random_state=42).reset_index(drop=True)

In [ ]:
# 4. Text Cleaning Function
def clean_text(text):
    text = text.lower() # Lowercase
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # Remove URLs
    text = re.sub(r'\@\w+|\#', '', text) # Remove mentions/hashtags
    text = re.sub(r'[^a-zA-Z\s]', '', text) # Remove special chars/numbers
    text = re.sub(r'\s+', ' ', text).strip() # Remove extra whitespace
    return text

df['clean_text'] = df['text'].apply(clean_text)

In [ ]:
df.head()

In [ ]:
# 5. Label Encoding
le = LabelEncoder()
df['label'] = le.fit_transform(df['subreddit'])
num_labels = len(le.classes_)
print(f"Classes to predict: {le.classes_}")

In [ ]:
# Split Data
X_train_text, X_test_text, y_train, y_test = train_test_split(
    df['clean_text'], df['label'], test_size=0.2, random_state=42, stratify=df['label']
)

# ==========================================
# 3. FEATURE ENGINEERING (Traditional Models)
# ==========================================

In [ ]:

print("Feature Engineering for Traditional Models...")
# TF-IDF Vectorization for XGBoost and SVM
tfidf = TfidfVectorizer(max_features=5000, stop_words='english')
X_train_tfidf = tfidf.fit_transform(X_train_text).toarray()
X_test_tfidf = tfidf.transform(X_test_text).toarray()

In [ ]:
X_train_tfidf

# ==========================================
# 4. MODEL 1: BERT (Deep Learning)
# ==========================================

In [ ]:
print("Training Model 1: BERT...")

# Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class RedditDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
# Prepare datasets
train_dataset = RedditDataset(X_train_text.tolist(), y_train.tolist(), tokenizer)
test_dataset = RedditDataset(X_test_text.tolist(), y_test.tolist(), tokenizer)

In [ ]:
# Load Model
model_bert = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=num_labels
).to(device)

In [ ]:
# Corrected Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)

In [ ]:
# Trainer
trainer = Trainer(
    model=model_bert,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [ ]:
trainer.train()

# Get BERT Predictions (Probabilities)
print("Generating BERT predictions...")
bert_preds_output = trainer.predict(test_dataset)
prob_bert = torch.nn.functional.softmax(torch.tensor(bert_preds_output.predictions), dim=-1).numpy()

# ==========================================
# 5. MODEL 2: XGBoost
# ==========================================

In [ ]:
print("Training Model 2: XGBoost...")
# XGBoost requires numerical input (provided by TF-IDF)
xgb_model = XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    use_label_encoder=False,
    eval_metric='mlogloss'
)

In [ ]:
xgb_model.fit(X_train_tfidf, y_train)

In [ ]:
# Get XGBoost Predictions (Probabilities)
prob_xgb = xgb_model.predict_proba(X_test_tfidf)

# ==========================================
# 6. MODEL 3: SVM
# ==========================================

In [ ]:
print("Training Model 3: SVM...")
# SVM needs probability=True to output class probabilities for the ensemble
svm_model = SVC(kernel='linear', probability=True, random_state=42)
svm_model.fit(X_train_tfidf, y_train)

In [ ]:
# Get SVM Predictions (Probabilities)
prob_svm = svm_model.predict_proba(X_test_tfidf)

# ==========================================
# 7. ENSEMBLE & EVALUATION
# ==========================================

In [ ]:
print("Combining models (Ensemble)...")

# Soft Voting Ensemble: Average the probabilities from all 3 models
# You can assign weights if one model is significantly better (e.g., 0.5*BERT + 0.25*XGB + 0.25*SVM)
avg_preds = (prob_bert + prob_xgb + prob_svm) / 3
final_predictions = np.argmax(avg_preds, axis=1)

In [ ]:
print("Average Prediction:\n ",avg_preds, "\nFinal predictions:\n", final_predictions)

In [ ]:
# Evaluation
print("\n=== Final Ensemble Classification Report ===")
print(classification_report(y_test, final_predictions, target_names=le.classes_))

In [ ]:
print(f"BERT Accuracy: {accuracy_score(y_test, np.argmax(prob_bert, axis=1)):.4f}")
print(f"XGBoost Accuracy: {accuracy_score(y_test, np.argmax(prob_xgb, axis=1)):.4f}")
print(f"SVM Accuracy: {accuracy_score(y_test, np.argmax(prob_svm, axis=1)):.4f}")
print(f"Ensemble Accuracy: {accuracy_score(y_test, final_predictions):.4f}")